# Databases, SQLAlchemy
## Objective
Build a movie recommendation service

Install `sqlalchemy`

In [11]:
!conda install -y sqlalchemy

Solving environment: done

## Package Plan ##

  environment location: /Users/suneelchakravorty/opt/anaconda3

  added / updated specs:
    - sqlalchemy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38hecd8cb5_1         2.9 MB
    ------------------------------------------------------------
                                           Total:         2.9 MB

The following packages will be UPDATED:

  conda                                4.9.2-py38hecd8cb5_0 --> 4.10.1-py38hecd8cb5_1



conda-4.10.1         | 2.9 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


## Explore the Open Brewery DB documentation
https://www.openbrewerydb.org/documentation/01-listbreweries#by_city

### Warm Up 1
Write a method search_breweries that searches breweries by some search term.

In [12]:
import requests

In [15]:
# requests.get('http://news.ycombinator.com').content

In [19]:
def search_breweries(search_term):
    url = 'https://api.openbrewerydb.org/breweries/search?query={}'.format(search_term)
    return requests.get(url).json()

In [21]:
records = search_breweries("dog")

In [23]:
len(records)

49

## Create our database

In [24]:
import sqlalchemy
sqlalchemy.__version__

'1.3.20'

In [31]:
from sqlalchemy import create_engine

In [34]:
engine = create_engine('sqlite:///openbrewery.db', echo = True)

Check if it exists.

In [35]:
conn = engine.connect()

2021-04-21 17:43:08,859 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-21 17:43:08,860 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:43:08,861 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-21 17:43:08,862 INFO sqlalchemy.engine.base.Engine ()


In [36]:
conn.close()

In [37]:
import os

In [38]:
os.listdir()

['Decorators.ipynb',
 '.DS_Store',
 'Generators.ipynb',
 'Iterators.ipynb',
 'Python and DBs.ipynb',
 'Packaging.ipynb',
 'openbrewery.db',
 'log.txt',
 'Metaclasses.ipynb',
 '.ipynb_checkpoints',
 'Review key intermediate Python concepts.ipynb']

Introspect `engine`

In [39]:
dir(engine)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_connection_cls',
 '_contextual_connect',
 '_echo',
 '_execute_clauseelement',
 '_execute_compiled',
 '_execute_default',
 '_execution_options',
 '_has_events',
 '_optional_conn_ctx_manager',
 '_run_visitor',
 '_should_log_debug',
 '_should_log_info',
 '_trans_ctx',
 '_wrap_pool_connect',
 'begin',
 'connect',
 'contextual_connect',
 'create',
 'dialect',
 'dispatch',
 'dispose',
 'driver',
 'drop',
 'echo',
 'engine',
 'execute',
 'execution_options',
 'get_execution_options',
 'has_table',
 'hide_parameters',
 'logger',
 'logging_name',
 'name',
 'pool',
 'raw_connection',
 'run_callable',
 'scalar',
 'schema_for_objec

List the table names.

In [40]:
engine.table_names()

2021-04-21 17:45:01,559 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-04-21 17:45:01,560 INFO sqlalchemy.engine.base.Engine ()


[]

## Defining a model
Let's define the City model.

[Here is a reference on data types](https://docs.sqlalchemy.org/en/14/core/type_basics.html)

In [41]:
import datetime

In [42]:

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, DateTime


Base = declarative_base()

In [43]:
class City(Base):
    __tablename__ = 'cities'

    city_id = Column(Integer, primary_key=True)
    name = Column(String(100))
    date_created = Column(DateTime, default=datetime.datetime.now)
    
    def __repr__(self):
        return "{}: {}".format(self.city_id, self.name)

Check the tables.

In [44]:
engine.table_names()

2021-04-21 17:47:45,073 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-04-21 17:47:45,077 INFO sqlalchemy.engine.base.Engine ()


[]

What s going on?

In [45]:
Base.metadata.create_all(engine)

2021-04-21 17:48:11,498 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cities")
2021-04-21 17:48:11,499 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:48:11,501 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("cities")
2021-04-21 17:48:11,502 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:48:11,507 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE cities (
	city_id INTEGER NOT NULL, 
	name VARCHAR(100), 
	date_created DATETIME, 
	PRIMARY KEY (city_id)
)


2021-04-21 17:48:11,511 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:48:11,520 INFO sqlalchemy.engine.base.Engine COMMIT


In [46]:
engine.table_names()

2021-04-21 17:48:12,006 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-04-21 17:48:12,008 INFO sqlalchemy.engine.base.Engine ()


['cities']

### Exercise
Define the BreweryType model based on the data available and create it.

- name

In [107]:
class BreweryType(Base):
    __tablename__ = 'brewerytypes'
    __table_args__ = {'extend_existing': True}
    
    brewery_type_id = Column(Integer, primary_key=True)
    name = Column(String(100))
    date_created = Column(DateTime, default=datetime.datetime.now)
    
    def __repr__(self):
        return "{}: {}".format(self.brewery_type_id, self.name)

In [51]:
Base.metadata

MetaData(bind=None)

In [49]:
Base.metadata.tables

immutabledict({'cities': Table('cities', MetaData(bind=None), Column('city_id', Integer(), table=<cities>, primary_key=True, nullable=False), Column('name', String(length=100), table=<cities>), Column('date_created', DateTime(), table=<cities>, default=ColumnDefault(<function datetime.now at 0x7fe0068b1d30>)), schema=None)})

In [52]:
Base.metadata.create_all(engine)

2021-04-21 17:51:29,942 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cities")
2021-04-21 17:51:29,949 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:51:29,962 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("brewerytypes")
2021-04-21 17:51:29,964 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:51:30,012 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("brewerytypes")
2021-04-21 17:51:30,022 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:51:30,026 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE brewerytypes (
	brewery_type_id INTEGER NOT NULL, 
	name VARCHAR(100), 
	date_created DATETIME, 
	PRIMARY KEY (brewery_type_id)
)


2021-04-21 17:51:30,027 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 17:51:30,031 INFO sqlalchemy.engine.base.Engine COMMIT


In [53]:
engine.table_names()

2021-04-21 17:51:33,013 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-04-21 17:51:33,014 INFO sqlalchemy.engine.base.Engine ()


['brewerytypes', 'cities']

## Creating instances and saving

In [56]:
from sqlalchemy.orm import sessionmaker

In [57]:
Session = sessionmaker(bind=engine)

In [58]:
session = Session()

In [59]:
new_city = City(name='Some city')

In [60]:
session.add(new_city)

ORM = object relationship/relational manager

In [61]:
our_city = session.query(City).first()

2021-04-21 17:53:47,844 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-21 17:53:47,854 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 17:53:47,855 INFO sqlalchemy.engine.base.Engine ('Some city', '2021-04-21 17:53:47.854502')
2021-04-21 17:53:47,867 INFO sqlalchemy.engine.base.Engine SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities
 LIMIT ? OFFSET ?
2021-04-21 17:53:47,871 INFO sqlalchemy.engine.base.Engine (1, 0)


In [62]:
our_city.name

'Some city'

### Exercise
Create a script to create cities for all the cities returned by a search query of "ale"

In [63]:
# Get the records

# For each record, create a City

In [66]:
records = search_breweries("green")

In [68]:
len(records)

45

In [71]:
for record in records:
    session.add(City(name=record['city']))

Let's query.

In [72]:
all_cities = session.query(City).all()

2021-04-21 18:05:50,317 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,319 INFO sqlalchemy.engine.base.Engine ('Lewes', '2021-04-21 18:05:50.317850')
2021-04-21 18:05:50,321 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,323 INFO sqlalchemy.engine.base.Engine ('Bowling Green', '2021-04-21 18:05:50.321613')
2021-04-21 18:05:50,324 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,325 INFO sqlalchemy.engine.base.Engine ('Greensboro', '2021-04-21 18:05:50.324562')
2021-04-21 18:05:50,326 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,327 INFO sqlalchemy.engine.base.Engine ('Columbia', '2021-04-21 18:05:50.326749')
2021-04-21 18:05:50,328 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,330

2021-04-21 18:05:50,543 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,544 INFO sqlalchemy.engine.base.Engine ('Green Bay', '2021-04-21 18:05:50.543448')
2021-04-21 18:05:50,545 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,546 INFO sqlalchemy.engine.base.Engine ('Northport', '2021-04-21 18:05:50.545368')
2021-04-21 18:05:50,547 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,549 INFO sqlalchemy.engine.base.Engine ('Green Bay', '2021-04-21 18:05:50.547869')
2021-04-21 18:05:50,550 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,551 INFO sqlalchemy.engine.base.Engine ('North Conway', '2021-04-21 18:05:50.550203')
2021-04-21 18:05:50,552 INFO sqlalchemy.engine.base.Engine INSERT INTO cities (name, date_created) VALUES (?, ?)
2021-04-21 18:05:50,

In [75]:
for city in all_cities:
    print(city.name)

Some city
Lewes
Bowling Green
Greensboro
Columbia
Silverton
Gaithersburg
Greensboro
Crawfordsville
Westport
Morganton
Morganton
Olean
Ithaca
Huntsville
Tucson
Prosser
Jacksonville Beach
Le Claire
Lewis Center
Manson
Port Saint Lucie
Green Bay
Green Bay
Saint Petersburg
Orange
Greeley
Lincoln
San Diego
Asheville
Lakewood
Middleburgh
Pitman
Green Bay
Green Bay
Green Bay
Bowling Green
Kansas City
Green Bay
Green Bay
Northport
Green Bay
North Conway
Ellisville
Greer
Greer


## Querying / Filtering

Let's get order cuisines alphabetically.


In [4]:
for c in session.query(City).order_by(City.name)[:10]:
    print(c)

### Deduping
A common and practical use case.

First, let's get all the distict names.

In [81]:
distinct_names = [name for name in session.query(City.name).distinct()]

2021-04-21 18:07:47,256 INFO sqlalchemy.engine.base.Engine SELECT DISTINCT cities.name AS cities_name 
FROM cities
2021-04-21 18:07:47,260 INFO sqlalchemy.engine.base.Engine ()


In [82]:
distinct_names

[('Some city'),
 ('Lewes'),
 ('Bowling Green'),
 ('Greensboro'),
 ('Columbia'),
 ('Silverton'),
 ('Gaithersburg'),
 ('Crawfordsville'),
 ('Westport'),
 ('Morganton'),
 ('Olean'),
 ('Ithaca'),
 ('Huntsville'),
 ('Tucson'),
 ('Prosser'),
 ('Jacksonville Beach'),
 ('Le Claire'),
 ('Lewis Center'),
 ('Manson'),
 ('Port Saint Lucie'),
 ('Green Bay'),
 ('Saint Petersburg'),
 ('Orange'),
 ('Greeley'),
 ('Lincoln'),
 ('San Diego'),
 ('Asheville'),
 ('Lakewood'),
 ('Middleburgh'),
 ('Pitman'),
 ('Kansas City'),
 ('Northport'),
 ('North Conway'),
 ('Ellisville'),
 ('Greer')]

Let's dedupe.

**Dedupe strategy**
1. For each name, get all the records that match the name
2. Pick one to be the survivor and delete the others.

- session.delete(obj)
- session.query(City).filter(City.name == name)

In [93]:
distinct_names = [rec[0] for rec in distinct_names]

In [94]:
distinct_names

['Some city',
 'Lewes',
 'Bowling Green',
 'Greensboro',
 'Columbia',
 'Silverton',
 'Gaithersburg',
 'Crawfordsville',
 'Westport',
 'Morganton',
 'Olean',
 'Ithaca',
 'Huntsville',
 'Tucson',
 'Prosser',
 'Jacksonville Beach',
 'Le Claire',
 'Lewis Center',
 'Manson',
 'Port Saint Lucie',
 'Green Bay',
 'Saint Petersburg',
 'Orange',
 'Greeley',
 'Lincoln',
 'San Diego',
 'Asheville',
 'Lakewood',
 'Middleburgh',
 'Pitman',
 'Kansas City',
 'Northport',
 'North Conway',
 'Ellisville',
 'Greer']

In [96]:
for name in distinct_names:
    cities = session.query(City).filter(City.name == name)
    # If we have dupe cities
    if cities.count() > 1:
        # Pick a survivor
        survivor = cities[0]
        # The rest are dupes
        dupes = cities[1:]
        # Delete each dupe
        for obj in dupes:
            session.delete(obj)

2021-04-21 18:20:25,484 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities 
WHERE cities.name = ?) AS anon_1
2021-04-21 18:20:25,486 INFO sqlalchemy.engine.base.Engine ('Some city',)
2021-04-21 18:20:25,491 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities 
WHERE cities.name = ?) AS anon_1
2021-04-21 18:20:25,492 INFO sqlalchemy.engine.base.Engine ('Lewes',)
2021-04-21 18:20:25,502 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities 
WHERE cities.name = ?) AS anon_1
2021-04-21 18:20:25,503 INFO sqlalchemy.engine.base.Engine ('Bowling Green',)
2021-04-21 18:20:25,50

2021-04-21 18:20:25,629 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities 
WHERE cities.name = ?) AS anon_1
2021-04-21 18:20:25,631 INFO sqlalchemy.engine.base.Engine ('Manson',)
2021-04-21 18:20:25,636 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities 
WHERE cities.name = ?) AS anon_1
2021-04-21 18:20:25,638 INFO sqlalchemy.engine.base.Engine ('Port Saint Lucie',)
2021-04-21 18:20:25,642 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities 
WHERE cities.name = ?) AS anon_1
2021-04-21 18:20:25,644 INFO sqlalchemy.engine.base.Engine ('Green Bay',)
2021-04-21 18:20:2

In [97]:
# Validation Check
all_cities = session.query(City).all()
for city in all_cities:
    print(city.name)

2021-04-21 18:21:13,680 INFO sqlalchemy.engine.base.Engine DELETE FROM cities WHERE cities.city_id = ?
2021-04-21 18:21:13,687 INFO sqlalchemy.engine.base.Engine (46,)
2021-04-21 18:21:13,693 INFO sqlalchemy.engine.base.Engine SELECT cities.city_id AS cities_city_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities
2021-04-21 18:21:13,725 INFO sqlalchemy.engine.base.Engine ()
Some city
Lewes
Bowling Green
Greensboro
Columbia
Silverton
Gaithersburg
Crawfordsville
Westport
Morganton
Olean
Ithaca
Huntsville
Tucson
Prosser
Jacksonville Beach
Le Claire
Lewis Center
Manson
Port Saint Lucie
Green Bay
Saint Petersburg
Orange
Greeley
Lincoln
San Diego
Asheville
Lakewood
Middleburgh
Pitman
Kansas City
Northport
North Conway
Ellisville
Greer


Let's find which ones start with "A"

## Mini-Lab 1: Recommendations & Reviews
Let's build a recommendation engine for users.

### Step 1: Populate Brewery Types
Look at the docs.

In [100]:
brewery_types = [
    'micro', 'nano', 'regional', 'brewpub',
    'large', 'planning', 'bar', 'contract',
    'proprietor', 'closed']

for brewery_type in brewery_types:
    session.add(BreweryType(name=brewery_type))

In [108]:
session.query(BreweryType).all()

2021-04-21 19:02:25,479 INFO sqlalchemy.engine.base.Engine SELECT brewerytypes.brewery_type_id AS brewerytypes_brewery_type_id, brewerytypes.name AS brewerytypes_name, brewerytypes.date_created AS brewerytypes_date_created 
FROM brewerytypes
2021-04-21 19:02:25,482 INFO sqlalchemy.engine.base.Engine ()


[1: micro,
 2: nano,
 3: regional,
 4: brewpub,
 5: large,
 6: planning,
 7: bar,
 8: contract,
 9: proprietor,
 10: closed]

### Step 2: Table Creation
There are three new tables we need to create

- Brewery (with the relevant fields)
- User (with username and user_id and favorite type and their city)
- Recommendations (which links to brewery and user)

### Step 3: Create_User
Create a helper function that creates a user and also adds favorite_type preference

```python
>>> create_user(username='alberto', favorite_type='..', city='Boston')
```

### Step 4: Simple matchmaking algo
Pick 5 random breweries that match their preferences and save to the database.

In [ ]:
recommend_breweries_and_save_to_db(user)

### (Optional) Step 5: Don't repeat the last 3 recommendations
Need to query the DB in the algo.